In [83]:
# Don't care. It's just a command for gist update.
run(`gist -u https://gist.github.com/5904067e7e7e3d18ecec3d1c14f8f427 CRF_slides.ipynb`)

https://gist.github.com/5904067e7e7e3d18ecec3d1c14f8f427


You can access this notebook from [here](http://nbviewer.jupyter.org/gist/KeitaW/5904067e7e7e3d18ecec3d1c14f8f427).

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\newcommand{\argmin}{\mathop{\rm arg~min}\limits}
$$

## 5.4 条件つき確率場（Conditional ramdom fields）


In [1]:
using PyCall
using DataFrames
using ScikitLearn
using Base.Test
using Base.Iterators

In [2]:
@pyimport nltk.corpus as corpus
function load_tagged_dataset(;ndata=Int(1e+06))
    tagged_words = corpus.brown[:tagged_words]()
    sentences = []; tags_list = []
    words = []; tags = []
    for (word, tag) in tagged_words
        if word == "."
            # Separate sentence by "."
            push!(sentences, words); push!(tags_list, tags)
            words = []; tags = []
        else
            push!(words, lowercase.(word)); push!(tags, tag)
        end
    end
    return DataFrame(
        sentences = sentences,
        tags_list = tags_list)
end

load_tagged_dataset (generic function with 1 method)

In [12]:
tagged_words_org = load_tagged_dataset()
head(tagged_words_org)

,sentences,tags_list
1,"Any[""the"", ""fulton"", ""county"", ""grand"", ""jury"", ""said"", ""friday"", ""an"", ""investigation"", ""of"", ""atlanta's"", ""recent"", ""primary"", ""election"", ""produced"", ""``"", ""no"", ""evidence"", ""''"", ""that"", ""any"", ""irregularities"", ""took"", ""place""]","Any[""AT"", ""NP-TL"", ""NN-TL"", ""JJ-TL"", ""NN-TL"", ""VBD"", ""NR"", ""AT"", ""NN"", ""IN"", ""NP\$"", ""JJ"", ""NN"", ""NN"", ""VBD"", ""``"", ""AT"", ""NN"", ""''"", ""CS"", ""DTI"", ""NNS"", ""VBD"", ""NN""]"
2,"Any[""the"", ""jury"", ""further"", ""said"", ""in"", ""term-end"", ""presentments"", ""that"", ""the"", ""city"", ""executive"", ""committee"", "","", ""which"", ""had"", ""over-all"", ""charge"", ""of"", ""the"", ""election"", "","", ""``"", ""deserves"", ""the"", ""praise"", ""and"", ""thanks"", ""of"", ""the"", ""city"", ""of"", ""atlanta"", ""''"", ""for"", ""the"", ""manner"", ""in"", ""which"", ""the"", ""election"", ""was"", ""conducted""]","Any[""AT"", ""NN"", ""RBR"", ""VBD"", ""IN"", ""NN"", ""NNS"", ""CS"", ""AT"", ""NN-TL"", ""JJ-TL"", ""NN-TL"", "","", ""WDT"", ""HVD"", ""JJ"", ""NN"", ""IN"", ""AT"", ""NN"", "","", ""``"", ""VBZ"", ""AT"", ""NN"", ""CC"", ""NNS"", ""IN"", ""AT"", ""NN-TL"", ""IN-TL"", ""NP-TL"", ""''"", ""IN"", ""AT"", ""NN"", ""IN"", ""WDT"", ""AT"", ""NN"", ""BEDZ"", ""VBN""]"
3,"Any[""the"", ""september-october"", ""term"", ""jury"", ""had"", ""been"", ""charged"", ""by"", ""fulton"", ""superior"", ""court"", ""judge"", ""durwood"", ""pye"", ""to"", ""investigate"", ""reports"", ""of"", ""possible"", ""``"", ""irregularities"", ""''"", ""in"", ""the"", ""hard-fought"", ""primary"", ""which"", ""was"", ""won"", ""by"", ""mayor-nominate"", ""ivan"", ""allen"", ""jr.""]","Any[""AT"", ""NP"", ""NN"", ""NN"", ""HVD"", ""BEN"", ""VBN"", ""IN"", ""NP-TL"", ""JJ-TL"", ""NN-TL"", ""NN-TL"", ""NP"", ""NP"", ""TO"", ""VB"", ""NNS"", ""IN"", ""JJ"", ""``"", ""NNS"", ""''"", ""IN"", ""AT"", ""JJ"", ""NN"", ""WDT"", ""BEDZ"", ""VBN"", ""IN"", ""NN-TL"", ""NP"", ""NP"", ""NP""]"
4,"Any[""``"", ""only"", ""a"", ""relative"", ""handful"", ""of"", ""such"", ""reports"", ""was"", ""received"", ""''"", "","", ""the"", ""jury"", ""said"", "","", ""``"", ""considering"", ""the"", ""widespread"", ""interest"", ""in"", ""the"", ""election"", "","", ""the"", ""number"", ""of"", ""voters"", ""and"", ""the"", ""size"", ""of"", ""this"", ""city"", ""''""]","Any[""``"", ""RB"", ""AT"", ""JJ"", ""NN"", ""IN"", ""JJ"", ""NNS"", ""BEDZ"", ""VBN"", ""''"", "","", ""AT"", ""NN"", ""VBD"", "","", ""``"", ""IN"", ""AT"", ""JJ"", ""NN"", ""IN"", ""AT"", ""NN"", "","", ""AT"", ""NN"", ""IN"", ""NNS"", ""CC"", ""AT"", ""NN"", ""IN"", ""DT"", ""NN"", ""''""]"
5,"Any[""the"", ""jury"", ""said"", ""it"", ""did"", ""find"", ""that"", ""many"", ""of"", ""georgia's"", ""registration"", ""and"", ""election"", ""laws"", ""``"", ""are"", ""outmoded"", ""or"", ""inadequate"", ""and"", ""often"", ""ambiguous"", ""''""]","Any[""AT"", ""NN"", ""VBD"", ""PPS"", ""DOD"", ""VB"", ""CS"", ""AP"", ""IN"", ""NP\$"", ""NN"", ""CC"", ""NN"", ""NNS"", ""``"", ""BER"", ""JJ"", ""CC"", ""JJ"", ""CC"", ""RB"", ""JJ"", ""''""]"
6,"Any[""it"", ""recommended"", ""that"", ""fulton"", ""legislators"", ""act"", ""``"", ""to"", ""have"", ""these"", ""laws"", ""studied"", ""and"", ""revised"", ""to"", ""the"", ""end"", ""of"", ""modernizing"", ""and"", ""improving"", ""them"", ""''""]","Any[""PPS"", ""VBD"", ""CS"", ""NP"", ""NNS"", ""VB"", ""``"", ""TO"", ""HV"", ""DTS"", ""NNS"", ""VBN"", ""CC"", ""VBN"", ""IN"", ""AT"", ""NN"", ""IN"", ""VBG"", ""CC"", ""VBG"", ""PPO"", ""''""]"


In [13]:
numdata = Int(1e+03)
tagged_words = tagged_words_org[1:numdata, :]
X_train, X_test, y_train, y_test = ScikitLearn.CrossValidation.train_test_split(tagged_words[:sentences], tagged_words[:tags_list], test_size = 0.3);
X_train_, X_test_, y_train_, y_test_ =  collect(flatten(X_train)), collect(flatten(X_test)), collect(flatten(y_train)), collect(flatten(y_test));
X_train_[1:10]

10-element Array{String,1}:
 "along"   
 "with"    
 "a"       
 "director"
 ","       
 "the"     
 "city"    
 "should"  
 "provide" 
 "a"       

## Omake: small code for making one hot vector of your data

In [47]:
word2int = (uniq_words) -> Dict(word => i for (i, word) in enumerate(uniq_words));
int2word = (uniq_words) -> Dict(i => word for (i, word) in enumerate(uniq_words));

In [50]:
to_onehot = (vocab_size) -> function (int)
    onehotvec = zeros(vocab_size)
    onehotvec[int] = 1
    return onehotvec
end
@test to_onehot(3)(1) == [1, 0, 0]

Test Passed


In [79]:
to_onehot_data = (wordint_dict) -> function (words)
    vocab_size = length(wordint_dict); data_size = length(words)
    onehot_data = zeros(vocab_size, data_size) # each column corresponds to one data as julia stores elements in column-major order
    onehot_vec = to_one_hot(vocab_size) 
    for (col, word) in enumerate(words) onehot_data[:, col] = onehot_vec(wordint_dict[word]) end
    return(onehot_data)
end
test_data = ["a", "b", "c", "d"]; test_x = ["a", "b", "c"]; test_y = ["d"]
wordint_dict = test_data |> word2int
to_onehot_data(wordint_dict)(test_x)
@test to_onehot_data(wordint_dict)(test_x) == [1 0 0; 0 1 0; 0 0 1; 0 0 0]
@test to_onehot_data(wordint_dict)(test_y) == [0 0 0 1]'

Test Passed


In [81]:
uniq_words = tagged_words[:sentences] |> flatten |> collect |> unique
@show length(uniq_words);

length(uniq_words) = 4926


In [ ]:
wordint_dict = t

Let $D = \{ (\mathbf{x}^{(1)}, \mathbf{y}^{(1)}), (\mathbf{x}^{(2)}, \mathbf{y}^{(2)}), \dots, (\mathbf{x}^{(T)}, \mathbf{y}^{(T)})\}$ be training dataset where,
* $\mathbf{x}$: Data (In this context, it's words),
* $\mathbf{y}$: Label (In this context, it's tags).

CRF is a kind of log linear model, thus conditional probability $P(\mathbf{y}\mid \mathbf{x})$ is defined as follows,
$$
P(\mathbf{y}\mid \mathbf{x}) = \frac{1}{Z_{\mathbf{x}, \mathbf{w}}} \exp (\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y})) \\
= \frac{\exp (\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y}))}{\sum_{\mathbf{y}}\exp (\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y}))} 
$$
where $\mathbf{\phi}(\mathbf{x}, \mathbf{y}) = (\sum_t \phi_1(\mathbf{x}, y_t, y_{t-1}), \sum_t \phi_2(\mathbf{x}, y_t, y_{t-1}) \dots, \sum_t \phi_k(\mathbf{x}, y_t, y_{t-1}), \dots, \sum_t \phi_F(\mathbf{x}, y_t, y_{t-1}))^{\mathbf{T}}$ and $F$ be the number of dimensions of our feature.

Thus, we could evaluate the optimal tag assignment by solving the following problem,
$$
\mathbf{y}^* = \argmax_{\mathbf{y}} \frac{1}{Z_{\mathbf{x}, \mathbf{w}}} \exp (\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y})) \\
= \argmax_{\mathbf{y}} \mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y}) 
$$

Here, using an assumption: $\phi_k(\mathbf{x}, \mathbf{y}) = \sum_t \phi_k(\mathbf{x}, y_t, y_{t-1})$ end up with,
$$
\mathbf{y}^* = \argmax_{\mathbf{y}} \sum_t \mathbf{w} \cdot \mathbf{\phi}(\mathbf{x}, y_t, y_{t-1})
$$
You could use the Viterbi algorithm to solve it.

We may use the Viterbi algorithm for the estimation of $\mathbf{y}^*$ once we get an appropriate $\mathbf{w}$. So, how do we learn $\mathbf{w}$? $\rightarrow$ __the Forward-backward algorithm.__

### The Forward-backward algorithm

We want to maximize the likelyhood function $L(\mathbf{w}) = \sum_{(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \in D} \log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})$. It can be accomplished by using the Gradient Descent method,
$$
\mathbf{w}^{new} = \mathbf{w}^{old} + \epsilon \nabla_\mathbf{w} L(\mathbf{w}^{old})
$$
But, its calculation can be very expensive as the second term of the RHS, $$\nabla_\mathbf{w} L(\mathbf{w}) = \sum_{(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \in D} \left( 
\mathbf{\phi}(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) - \sum_{\mathbf{y}}P(\mathbf{y} \mid \mathbf{x}^{(i)})\mathbf{\phi}(\mathbf{x}^{(i)}, \mathbf{y})
\right)$$
requires calculation of every possible $P(\mathbf{y} \mid \mathbf{x}^{(i)})$ in terms of  $\mathbf{y}$.

Using the assumption yields,
$$
P(\mathbf{y} \mid \mathbf{x}^{(i)})\mathbf{\phi}(\mathbf{x}^{(i)}, \mathbf{y}) = \sum_{\mathbf{y}}P(\mathbf{y} \mid \mathbf{x}) \sum_t \mathbf{\phi}(\mathbf{x}, y_t, y_{t-1}) = \sum_t \sum_{y_t, y_{t-1}} P(y_{t-1}, y_t \mid \mathbf{x})\mathbf{\phi}(\mathbf{x}, y_t, y_{t-1})
$$
We'll discuss how to calculate $P(y_{t-1}, y_t \mid \mathbf{x})$ efficiently in the subsequent slides.

Let $T$ be the length of input $\mathbf{x}$. We define dummy labels $y_0$ and $y_{T+1}$ as $\mathbf{B}$ and $\mathbf{E}$ respectively.
We denote $\sum_{y_0, \cdots, y_{t-2}}$ as $\sum_{\mathbf{y}_{0:t-2}}$ and $\psi_t(y_t, y_{t-1})$ as $\exp(\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, y_t, y_{t-1}))$ to simplify the notation. 

$$
P(y_{t-1}, y_t \mid \mathbf{x}) = \sum_{\mathbf{y}_{0:t-2}} \sum_{\mathbf{y}_{t+1:t-2}} P(\mathbf{y} \mid \mathbf{x}) \\
 = \frac{1}{Z_{\mathbf{x}, \mathbf{w}}} \sum_{\mathbf{y}_{0:t-2}} \sum_{\mathbf{y}_{t+1:t-2}} \prod_{t'} \psi_{t'}(y_{t'}, y_{t'-1}) \\
 = \frac{\psi_{t'}(y_{t'}, y_{t'-1})}{Z_{\mathbf{x}, \mathbf{w}}} \sum_{\mathbf{y}_{0:t-2}} \sum_{\mathbf{y}_{t+1:t-2}} \prod_{t'\neq t} \psi_{t'}(y_{t'}, y_{t'-1}) \\
 = \frac{\psi_{t'}(y_{t'}, y_{t'-1})}{Z_{\mathbf{x}, \mathbf{w}}} \left(\sum_{\mathbf{y}_{0:t-2}} \prod_{t'=1}^{t-1} \psi_{t'}(y_{t'}, y_{t'-1}) \right) \left(\sum_{\mathbf{y}_{t+1:t-2}} \prod_{t'= t+1}^{T} \psi_{t'}(y_{t'}, y_{t'-1})\right) \\
 = \frac{\psi_{t'}(y_{t'}, y_{t'-1})}{Z_{\mathbf{x}, \mathbf{w}}} \alpha(y_t, t)\beta(y_t, t)
$$
where
$$
\alpha(y_t, t) = \sum_{\mathbf{y}_{0:t-2}} \prod_{t'=1}^{t-1} \psi_{t'}(y_{t'}, y_{t'-1})\\
\beta(y_t, t) = \sum_{\mathbf{y}_{t+1:t-2}} \prod_{t'= t+1}^{T} \psi_{t'}(y_{t'}, y_{t'-1})
$$

We could calculate efficiently $\alpha(y_t, t)$, $\beta(y_t, t)$ because,
$$
\alpha(y_t, t) = \sum_{\mathbf{y}_{0:t-1}} \prod_{t'=1}^{t} \psi_{t'}(y_{t'}, y_{t'-1}) \\
 = \sum_{y_{t-1}}\psi_{t}(y_t, y_{t-1})\sum_{\mathbf{y}_{0:t-2}}\prod_{t'=1}^{t-1} \psi_{t'}(y_{t'}, y_{t'-1}) \\
 =  \sum_{y_{t-1}}\psi_{t}(y_t, y_{t-1})\alpha(y_{t-1}, t-1)
$$
$$
\beta(y_t, t) = \sum_{y_{t+1}}\psi_{t+1}(y_{t+1}, y_{t})\beta(y_{t+1}, t+1)
$$
Also note that,
$$
Z_{\mathbf{x}, \mathbf{w}} = \sum_{\mathbf{y}}\exp (\mathbf{w}\cdot \mathbf{\phi}(\mathbf{x}, \mathbf{y})) = \sum_{\mathbf{y}_{0, T-2}} \prod_{t=1}^{T-1} \psi_t(y_t, y_{t-1}) = \sum_{y_T} \alpha(y_T, T)
$$

Here we'll use a $F$-dimensional simple feature $\mathbf{\phi}(\mathbf{x}, y_t, y_{t-1})$ which $k$-th element satisfies the following equation,
$$
\phi_k(\mathbf{x}, y_t, y_{t-1}) = \begin{cases}
1 \quad \text{Ether $y_t$ or $y_{t-1}$ is the $k$-th tag of the unique tag list}\\
0 \quad \text{Otherwise}
\end{cases}
$$
where $y_t$ and $y_{t-1}$ are the corresponding tag of $x_t$ and $x_{t-1}$ respectively.

__Sorry I couldn't implement the algorithm in time! I'll inform you after I make it.__